
Imports

In [110]:
import tensorflow as tf
from tensorflow import keras
import time

Carregamento das funções que nos darão os modelos para cada topologia de rede.

In [111]:
# Crie o modelo de rede neural convolucional simples
def get_cifar10_network(num_classes,input_shape):
    model = keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),#(32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')  # 10 classes de saída
    ])

    # Compile o modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                  metrics=['accuracy'])

    model.summary()

    return model

def get_fashion_mnist_network(num_classes,input_shape):
  model = keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),#(32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),  # Aumentando para 64 filtros
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(num_classes, activation='softmax')  # 10 classes de saída
  ])

  # Compile o modelo
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                metrics=['accuracy'])

  model.summary()

  return model
  #pass  # substitua pelo seu codigo

def get_mnist_network(num_classes,input_shape):
  model = keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),  # Usando 32 filtros
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),  # Aumentando para 64 filtros
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax') 
  ])

  # Compile o modelo
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                metrics=['accuracy'])

  model.summary()

  return model

def get_cifar100_network(num_classes,input_shape):
  model = keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),#(32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),  # Aumentando para 64 filtros
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),  # Aumentando para 64 filtros
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # 10 classes de saída
  ])

  # Compile o modelo
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                metrics=['accuracy'])

  model.summary()

  return model

In [112]:
model_functions = [get_mnist_network, get_fashion_mnist_network, get_cifar10_network, get_cifar100_network]

In [113]:
topologies = [
    {
        "name": 'mnist',
        "input_shape": (28,28,1),
        "num_classes": 10
    },
    {
        "name": 'fashion_mnist',
        "input_shape": (28,28,1),
        "num_classes": 10
    },
    {
        "name": 'cifar10',
        "input_shape": (32,32,3),
        "num_classes": 10
    },
    {
        "name": 'cifar100',
        "input_shape": (32,32,3),
        "num_classes": 100
    }
]

Carregamento de cada dataset e treinamento

In [125]:
#deixando a classe de
results = {}

In [126]:
# Carregar o conjunto de dados CIFAR-10
for tp in topologies:
    print("-----------------------")
    print(f"using {tp['name']} dataset")
    dataset = getattr(keras.datasets, tp['name'])
    results[tp['name']] = {}
    for function in model_functions:
        print(f"training using {function.__name__} model with {tp['name']} dataset\n")    
        #Carrega duas tuplas, representando os dados de treinamento e de teste.
        #Cada tupla tem as imagens e os respectivos rótulos
        (train_images, train_labels), (test_images, test_labels) = dataset.load_data()
        num_classes = tp['num_classes']
        # Converter para codificação one-hot dos labels
        train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)
        test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)
        #Não é necessário se utilizar como função de custo esparse_categorical_cross_entropy

        model = function(num_classes=num_classes, input_shape=tp['input_shape'])
        
        inicio = time.time()
        model.fit(train_images, train_labels, epochs=10)
        fim = time.time()

        # Avalie o modelo no conjunto de teste
        test_loss, test_accuracy = model.evaluate(test_images, test_labels)
        print(f'test accuracy: {test_accuracy * 100:.2f}%')
        
        results[tp['name']][f'{function.__name__}'] = {'time': (fim - inicio), 'acc': f'{test_accuracy * 100:.2f}%'}
        print("finishing training")
        print("~~~~~~~~~~~~~~~~~~")
    print("finishing using dataset")
    print("-----------------------\n")
    del dataset

-----------------------
using mnist dataset
training using get_mnist_network model with mnist dataset



d:\UFRGS\ETAPA 6\ia\trabalhos_praticos_IA\tensorflow\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_87 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_87 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_88 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_88 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_44 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_88 (Dense)                │ (None, 64)             │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_89 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121,930 (476.29 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_100 (Conv2D)             │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_100               │ (None, 13, 13, 32)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_101 (Conv2D)             │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_101               │ (None, 5, 5, 64)       │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_102 (Conv2D)             │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_102               │ (None, 1, 1, 128)      │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_51 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_102 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_103 (Dense)               │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,578 (396.79 KB)

 Trainable params: 101,578 (396.79 KB)

 Non-trainable params: 0 (0.00 B)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.7390 - loss: 1.1663
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8451 - loss: 0.4378
test accuracy: 84.40%
finishing training
~~~~~~~~~~~~~~~~~~
finishing using dataset
-----------------------



Preencha o dict abaixo substituindo os None com a acuracia final (acc) e o tempo de treinamento (time) encontrado no seu experimento pra cada dataset

In [127]:
results

{'mnist': {'get_mnist_network': {'time': 9.979483366012573, 'acc': '96.93%'},
  'get_fashion_mnist_network': {'time': 13.865436553955078, 'acc': '97.72%'},
  'get_cifar10_network': {'time': 10.405070543289185, 'acc': '96.45%'},
  'get_cifar100_network': {'time': 11.531903743743896, 'acc': '97.12%'}},
 'fashion_mnist': {'get_mnist_network': {'time': 10.215243577957153,
   'acc': '85.34%'},
  'get_fashion_mnist_network': {'time': 13.01742148399353, 'acc': '86.83%'},
  'get_cifar10_network': {'time': 8.38051462173462, 'acc': '87.32%'},
  'get_cifar100_network': {'time': 11.918577432632446, 'acc': '84.40%'}}}

In [128]:
a = {}
a['oi']["mnist"] = 10
a['oi']["fashionmnist"] = 10
a

KeyError: 'oi'